<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Biderectional_LSTM_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on Google Colab with Pytorch Version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>Please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.981)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time formating please wait...\n>>>time elapse is: {time_fmt()}")

Mounted at /content/drive/
You are on Google Colab with Pytorch Version: 1.8.1+cu101
>>>time formating please wait...
>>>time elapse is: 0: 02: 03.00


In [2]:
#In this notbook we are going to train a biderectional rnn with an LSTM architecture on cifar10 dataset
#here cifar10 is treated as a sequence data with 32 feature's size and 32 sequence legnth: We will ignore the channel dimension

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from tqdm import tqdm
import time, sys

In [37]:
#Hyperparameters for the model:
batch_size = 64
EPOCHS = 10
learning_rate = 1e-3
sequence_length = 32
input_size = 32
hidden_size = 512
num_layers = 2
num_classes = 10

In [27]:
#BLSTM-class: This class build the bi-derectional-rnn with 2 layers LSTM and a fc output layer:

In [38]:
class BRNN(nn.Module):
  def __init__(self,input_size, hidden_size, num_layers, num_classes):
    super(BRNN, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.brnn = nn.LSTM(input_size, 
                        hidden_size,
                        num_layers, 
                        batch_first = True,
                        bidirectional = True)
    self.fc = nn.Linear(hidden_size*2, num_classes)
    
  def forward(self, input_tensor):
    #initialize the hidden and cell states
    c0 = torch.rand(self.num_layers * 2, input_tensor.size(0), self.hidden_size).to(device = device)
    h0 = torch.rand(self.num_layers * 2, input_tensor.size(0), self.hidden_size).to(device = device)
    output,_ = self.brnn(input_tensor, (h0,c0))
    output = output[:,-1,:]
    return self.fc(output)

In [39]:
#Instantiating and test the model class to see if its deliver the desired shape:(batch,10)
model = BRNN(input_size, hidden_size,num_layers,num_classes).to(device = device)
rnd_data = torch.rand(64, 3,32,32)[:,-1,:].to(device = device)
print(f"output_shape: {model(rnd_data).shape}")

output_shape: torch.Size([64, 10])


In [40]:
#Load and split the data into batches with 64(minibatch) sizes the data from torch
train_data = datasets.CIFAR10(root = '/train_data', train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.CIFAR10(root = '/test_data', train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(dataset = test_data, shuffle = True, batch_size = batch_size)
x_batch, y_batch = next(iter(train_loader))
print(f"x_batch_shape: {x_batch.shape}\ty_batch_shape: {y_batch.shape}")

Files already downloaded and verified
Files already downloaded and verified
x_batch_shape: torch.Size([64, 3, 32, 32])	y_batch_shape: torch.Size([64])


In [41]:
#Get the loss and optimizers objects
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [42]:
#Training loop from scatch:
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>Train starts for epoch: {epoch + 1}\n>>>Please MAITAIN SCREEN ACTIVITY!!!\n>>>Model is training: Please wait.....")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    #Loading the data into GPU when available
    data = data[:,-1,:].to(device = device)#Alse we squeeze the channel dimension
    target = target.to(device = device)
    #Forward pass here
    preds = model(data)
    train_loss = loss_fn(preds, target)
    #Backward pass here
    optimizer.zero_grad()
    train_loss.backward()
    #Gradient descent with Adam optimizer
    optimizer.step()

#Evaluating model's performance:
def performance_eval(loader, model):
  if loader.dataset.train:
    print(f"\n>>>Checking the performance over the trainining data\n>>>Please wait...")
  else:
    print(f"\n>>>Checking the performance over the validation data\n>>>Please wait...")
  num_correct = 0
  num_examples = 0
  model.eval()
  #No need to compute the slopr this time:
  with torch.no_grad():
    for x, y in(loader):
      x = x[:,-1,:].to(device = device)
      y = y.to(device = device)
      preds = model(x)
      _,predictions = preds.max(1)#fetch maximum probability in the vector of 10 proba-values
      num_correct+=(predictions == y).sum()#Get the total for all correct classified samples
      num_examples+=predictions.size(0)#Fetch all examples in a batch (64 in this case)
  model.train()
  return float(num_correct/num_examples)*100 #compute the accuracy (in percentage)

toc = time.time()
print(f"\n>>>The accuracy over the training dataset is: {performance_eval(train_loader, model):.2f}")
print(f"\n>>>The accuracy over the validation dataset is: {performance_eval(test_loader, model):.2f}")
print(f"\n>>>Total time elspsed for the training and validation is: {time_fmt(toc - tic)}")

  1%|          | 5/782 [00:00<00:19, 40.39it/s]


>>>Train starts for epoch: 1
>>>Please MAITAIN SCREEN ACTIVITY!!!
>>>Model is training: Please wait.....


  1%|          | 5/782 [00:00<00:18, 43.12it/s]


>>>Train starts for epoch: 2
>>>Please MAITAIN SCREEN ACTIVITY!!!
>>>Model is training: Please wait.....


  1%|          | 5/782 [00:00<00:17, 43.41it/s]

...


>>>Train starts for epoch: 9
>>>Please MAITAIN SCREEN ACTIVITY!!!
>>>Model is training: Please wait.....


  1%|          | 5/782 [00:00<00:18, 42.26it/s]


>>>Train starts for epoch: 10
>>>Please MAITAIN SCREEN ACTIVITY!!!
>>>Model is training: Please wait.....


100%|██████████| 782/782 [00:19<00:00, 40.54it/s]



>>>Checking the performance over the trainining data
>>>Please wait...

>>>The accuracy over the training dataset is: 86.69

>>>Checking the performance over the validation data
>>>Please wait...

>>>The accuracy over the validation dataset is: 56.86

>>>Total time elspsed for the training and validation is: 0: 03: 13.00
